# Replacing ADC table by L1 distance between PQ codes

In [10]:
using HDF5
using BenchmarkTools
using Distances
using LoopVectorization
using SIMD
using Clustering
using ProgressMeter
using StaticArrays
using DataFrames
using Plots
using NPZ

In [11]:
profile_flag = false
Sys.cpu_info()[1].model

"Apple M1 Pro"

In [12]:
#path = joinpath(homedir(), "TFM", "ann-benchmarks",  "sift-128-euclidean.hdf5")
path = joinpath(homedir(), "Datasets", "SIFT1M", "sift-128-euclidean.hdf5")

"/Users/dbuchaca/Datasets/SIFT1M/sift-128-euclidean.hdf5"

In [13]:
f = h5open(path, "r")

X_tr_vecs = read(f["train"])
X_te_vecs = read(f["test"]);
true_neighbors = read(f["neighbors"])
true_distances = read(f["distances"])

true_neighbors .= true_neighbors .+ 1;

@show size(X_tr_vecs)
@show size(X_te_vecs)
@show size(true_neighbors)
@show size(true_distances)

size(X_tr_vecs) = (128, 1000000)
size(X_te_vecs) = (128, 10000)
size(true_neighbors) = (100, 10000)
size(true_distances) = (100, 10000)


(100, 10000)

### Compute Recall PQLinearScann 

In [14]:
function recall(predicted, relevant, eval_at)
    """
    fraction of the relevant documents that are successfully retrieved
    """
    if eval_at == 0
        return 0.0
    end
    
    predicted_at_k = predicted[1:eval_at]
    n_predicted_and_relevant =  length(intersect( Set(predicted_at_k), Set(relevant))) 
    return n_predicted_and_relevant/ length(relevant)
end

recall (generic function with 1 method)

##  PQLinearscann Sharing prototypes across features

https://groups.google.com/g/julia-users/c/xBcQRebyi_o



In [15]:
n_features, n_examples = size(X_tr_vecs)

function encode_shared(dist, vector::Array{T}, shared_prototypes::Array{T}) where T
    n_clusters = length(shared_prototypes)
    closest_prototypes = Array{Int32}(undef, n_features, 1);
    
    @inbounds for (j,x) in enumerate(vector)
        best_coordinate = 1
        min_distance::T = typemax(T)
        for k in 1:n_clusters
           current_dist = dist(shared_prototypes[k], x)
           if current_dist < min_distance
               best_coordinate = k
               min_distance = current_dist
           end
           #println(k, ' ', j, ' ', best_coordinate, ' ',min_distance )
        end            
        closest_prototypes[j] = best_coordinate
    end
    return closest_prototypes
end

encode_shared (generic function with 1 method)

We load the K=32 centroids resulting of performing 1d-kmeans over the first feature of the train dataset. Notice that these centroids are sorted, resulting in a sorted codification. We will take advantage of shared quantization.

In [16]:
P_shared = vec(Float32.(npzread("1dkmeans_shared_prototypes.npy")))

32-element Vector{Float32}:
   0.22927776
   2.4602568
   4.935903
   7.958296
  10.961814
  13.964964
  16.974878
  19.983488
  23.455843
  27.45342
  31.461033
  35.46336
  39.47896
   ⋮
  76.971436
  82.43247
  88.45911
  94.42536
 100.457985
 106.51546
 112.64925
 118.534
 124.41393
 130.70255
 138.0879
 148.98564

In [17]:
PQcodes_shared = Array{Int8}(undef, n_features, n_examples);

for j in 1:n_examples
    PQcodes_shared[:,j] = encode_shared(euclidean, X_tr_vecs[:,j], P_shared)  
end


### Inspect idea of computing distances without adc table: directly from pqcodes


In [45]:
function abs_dist(y::Array{T}, X::Array{T}, j) where T
    # Here I use a bigger Int type than 8 due to avoid
    # res beeing overflowed
    res = Int16(0)
    @inbounds @fastmath  for k in eachindex(y)
        res += abs(X[k, j] - y[k])
    end
    return res
end

function linear_scann_exact(dist, query, X)

    n_features, n_examples = size(X)
    distances = Array{Float32}(undef, n_examples)
    
    @inbounds for j in 1:n_examples
        distances[j] = dist(query, X, j)    
    end
    return distances
end

linear_scann_exact (generic function with 1 method)

There is some problem with the casting of query_code, probably related with the fact that "res" adopts the same type than the query_code on abs_dist:

In [46]:
query_id = 1
query = X_te_vecs[:,query_id];
query_true_neighbors = true_neighbors[:,query_id]
top_k = 100

query_code = encode_shared(euclidean, query, P_shared)
query_code = Int8.(vec(query_code))

PQcodes_int8 = Int8.(PQcodes_shared);

pq_distances = linear_scann_exact(abs_dist, query_code, PQcodes_int8)
top_k_pq = sortperm(pq_distances)[1:top_k];

@show recall(top_k_pq, query_true_neighbors, top_k);

recall(top_k_pq, query_true_neighbors, top_k) = 0.58


In [20]:
@benchmark linear_scann_exact($abs_dist, $query_code, $PQcodes_int8)

BenchmarkTools.Trial: 867 samples with 1 evaluation.
 Range (min … max):  5.353 ms … 24.298 ms  ┊ GC (min … max): 0.00% … 76.01%
 Time  (median):     5.610 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.761 ms ±  1.307 ms  ┊ GC (mean ± σ):  1.99% ±  6.35%

                   ▇█▇▇▃                                      
  ▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▂▅█████▆▅▅▃▅▆▅▄▄▄▃▃▄▃▃▃▃▂▃▃▂▂▃▃▂▂▂▂▂▁▂▂▁▁▂▂ ▃
  5.35 ms        Histogram: frequency by time        6.08 ms <

 Memory estimate: 3.81 MiB, allocs estimate: 2.

## Top_k_expansion + finetunning

An interesting idea would be to compute a candidate space of nearest neighbours and "finetunit" on using exact distances.

Here he have an `extra_factor` that can be used to tune the top_k expansion (the higher the better quality at the expense of time).

In [145]:
function linear_scann_exact_pq(dist, query, X, top_k, extra_factor)

    n_features, n_examples = size(X)
    distances = Array{Float32}(undef, n_examples)
    
    @inbounds for j in 1:n_examples
        distances[j] = dist(query, X, j)    
    end

    top_result_pos = sortperm(distances)[1:top_k*extra_factor];
    
    return top_result_pos
end

query_id = 1
query = X_te_vecs[:,query_id];
query_true_neighbors = true_neighbors[:,query_id]
top_k = 100

query_code = encode_shared(euclidean, query, P_shared)
query_code = Int8.(vec(query_code))

PQcodes_int8 = Int8.(PQcodes_shared);
best_ids = linear_scann_exact_pq(abs_dist, query_code, PQcodes_int8, top_k, 10);

@show recall(best_ids[1:top_k], query_true_neighbors, top_k);

recall(best_ids[1:top_k], query_true_neighbors, top_k) = 0.58


In [131]:
function euclidean_mat2(y, X, j) where T
    # Here I use a bigger Int type than 8 due to avoid
    # res beeing overflowed
    res = zero(eltype(y))
    @inbounds @fastmath  for k in eachindex(y)
        partial = X[k, j] - y[k]
        res += partial * partial
    end
    return res
end

function linear_scann_exact(dist, query, X)

    n_features, n_examples = size(X)
    distances = Array{Float32}(undef, n_examples)
    
    @inbounds for j in 1:n_examples
        distances[j] = dist(query, X, j)    
    end
    return distances
end

linear_scann_exact (generic function with 1 method)

We could compute exact distances within a subset of examples

In [140]:

query_id = 5
query = X_te_vecs[:,query_id];
query_true_neighbors = true_neighbors[:,query_id]
top_k = 100

query_code = encode_shared(euclidean, query, P_shared)
query_code = Int8.(vec(query_code))

PQcodes_int8 = Int8.(PQcodes_shared);
best_ids = linear_scann_exact_pq(abs_dist, query_code, PQcodes_int8, top_k, 10);


#linear_scann_exact(euclidean_mat, query, view(X_tr_vecs,:,best_ids))
distances_candidates_expanded = linear_scann_exact(euclidean_mat2, query, X_tr_vecs[:,best_ids]);
permutation_expanded = sortperm(distances_candidates_expanded)[1:top_k];
@show recall(best_ids[permutation_expanded], query_true_neighbors, top_k);

recall(best_ids[permutation_expanded], query_true_neighbors, top_k) = 0.99


Here the problem is that we assume X_tr_vecs is "on memory" and we want to avoid this as much as possible,
because it can be potentially quite big. We have to investigate how to store the 'exact values from X_tr_vecs' on disk, 
using a memmap array like storage. Also study the overhead of doing this.

In [133]:
@benchmark distances_candidates_expanded = linear_scann_exact(euclidean_mat2, query, X_tr_vecs[:,best_ids])

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  118.542 μs …  16.981 ms  ┊ GC (min … max): 0.00% … 98.16%
 Time  (median):     140.250 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   157.080 μs ± 479.008 μs  ┊ GC (mean ± σ):  9.97% ±  3.24%

                               █▇▅▂▁▁▁                           
  ▂▂▂▂▂▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▂▃█████████▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂ ▃
  119 μs           Histogram: frequency by time          159 μs <

 Memory estimate: 527.39 KiB, allocs estimate: 1493.

Other test with hamming



In [143]:

@inline function hamming(y::Array{T}, X::Array{T}, j) where T
    # Here I use a bigger Int type than 8 due to avoid
    # res beeing overflowed
    res = UInt8(0)
    @inbounds @simd  for k in eachindex(y)
        res += X[k, j] != y[k]
    end
    return res
end

function linear_scann_exact_hamming( query, X)

    n_features, n_examples = size(X)
    distances = Array{UInt8}(undef, n_examples)
    
    @inbounds for j in 1:n_examples
        distances[j] = hamming(query, X, j)    
    end
    return distances
end

linear_scann_exact_hamming (generic function with 1 method)

In [144]:
query_id = 20
query = X_te_vecs[:,query_id];
query_true_neighbors = true_neighbors[:,query_id]
top_k = 100

query_code = encode_shared(euclidean, query, P_shared)
query_code = UInt8.(vec(query_code))

PQcodes_int8 = UInt8.(PQcodes_shared);

pq_distances = linear_scann_exact(hamming, query_code, PQcodes_int8)
top_k_pq = sortperm(pq_distances)[1:top_k];

@show recall(top_k_pq, query_true_neighbors, top_k);

recall(top_k_pq, query_true_neighbors, top_k) = 0.01


In [42]:
@benchmark linear_scann_exact_hamming($query_code, $PQcodes_int8)

BenchmarkTools.Trial: 1553 samples with 1 evaluation.
 Range (min … max):  3.069 ms …  22.735 ms  ┊ GC (min … max): 0.00% … 85.91%
 Time  (median):     3.175 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.208 ms ± 637.222 μs  ┊ GC (mean ± σ):  0.84% ±  3.54%

          ▁▁            ▅▇██▇▆▅▅▅▃▃▃▁▂▁                       ▁
  ▅▁▁▅▄▁▄▁███▇▆▅▆▅▆▄▄▄▄█████████████████▇▇▆▆▆▇▇▅▅▅▆▅▆▇▄▄▆▅▅▄▆ █
  3.07 ms      Histogram: log(frequency) by time      3.31 ms <

 Memory estimate: 976.67 KiB, allocs estimate: 2.

All the top k distances are 0. This would be the expected output if all top_k_pq codes were the same and also equal to the query code (NOT TRUE):

In [31]:
println(PQcodes_shared[:,top_k_pq[1]])
println(PQcodes_shared[:,top_k_pq[2]])
println(encode_shared(euclidean, query, P_shared))

Int8[1, 2, 8, 13, 3, 9, 8, 1, 1, 10, 12, 3, 2, 12, 19, 1, 2, 6, 5, 1, 1, 7, 26, 9, 3, 1, 1, 1, 1, 2, 8, 9, 1, 1, 1, 12, 24, 20, 18, 2, 1, 1, 2, 11, 9, 25, 29, 1, 20, 1, 1, 1, 1, 12, 29, 25, 28, 5, 1, 1, 1, 1, 4, 16, 5, 1, 4, 21, 15, 7, 6, 9, 3, 1, 19, 29, 8, 5, 7, 5, 29, 11, 13, 26, 2, 2, 4, 12, 29, 12, 1, 1, 2, 4, 1, 5, 8, 1, 2, 7, 11, 5, 5, 12, 19, 9, 11, 24, 4, 2, 4, 14, 8, 17, 29, 26, 1, 1, 1, 1, 13, 15, 8, 2, 4, 4, 1, 1]
Int8[5, 2, 2, 15, 12, 14, 9, 1, 16, 7, 3, 14, 10, 15, 18, 18, 6, 3, 1, 1, 7, 21, 25, 19, 8, 1, 1, 1, 2, 6, 10, 23, 1, 1, 1, 5, 26, 22, 16, 6, 1, 1, 1, 10, 27, 24, 22, 8, 20, 3, 1, 9, 7, 13, 27, 25, 27, 9, 3, 7, 6, 6, 13, 20, 3, 1, 1, 11, 16, 10, 21, 12, 5, 2, 9, 25, 20, 3, 5, 6, 24, 13, 22, 21, 6, 1, 1, 5, 27, 12, 5, 10, 13, 4, 7, 10, 7, 2, 8, 20, 9, 3, 3, 5, 24, 10, 16, 19, 4, 1, 1, 6, 9, 8, 18, 20, 10, 3, 2, 8, 7, 4, 5, 3, 6, 6, 11, 8]
Int32[1; 2; 5; 27; 18; 9; 3; 1; 14; 8; 9; 7; 3; 10; 18; 4; 5; 1; 1; 1; 1; 13; 25; 8; 8; 2; 3; 2; 2; 4; 7; 12; 1; 1; 4; 9; 26; 28

This does work:

In [137]:
query_id = 1
query = X_te_vecs[:,query_id];
query_true_neighbors = true_neighbors[:,query_id]
top_k = 100

query_code = encode_shared(euclidean, query, P_shared)
#query_code = UInt32.(vec(query_codeode))

PQcodes_uint8 = Int32.(PQcodes_shared);

pq_distances = linear_scann_exact(abs_dist, query_code, PQcodes_uint8)
top_k_pq = sortperm(pq_distances)[1:top_k];

@show recall(top_k_pq, query_true_neighbors, top_k);

recall(top_k_pq, query_true_neighbors, top_k) = 0.58


## Benchmark times exact vs linearscann

#### Type UInt8

In [138]:
query_id = 1
query = X_te_vecs[:,query_id];
query_true_neighbors = true_neighbors[:,query_id]
top_k = 100

query_code = encode_shared(euclidean, query, P_shared)
query_code = UInt8.(vec(query_code))

PQcodes = UInt8.(PQcodes_shared);

In [139]:
@inline function Euclidean0(x, query)
    res = zero(eltype(x))
    @inbounds @fastmath  for j in eachindex(x)
        aux = (query[j] - x[j])
        res += aux * aux
    end
    return sqrt(res)
end

function linear_scann_exact(dist, query, X)

    n_features, n_examples = size(X)
    distances = Array{Float32}(undef, n_examples)
    
    @inbounds for j in 1:n_examples
        distances[j] = dist(query, view(X,:,j))    
    end
    return distances
end

@benchmark linear_scann_exact($Euclidean0, $query, $X_tr_vecs)


BenchmarkTools.Trial: 150 samples with 1 evaluation.
 Range (min … max):  27.112 ms … 39.131 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     33.514 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   33.495 ms ±  2.502 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

                                 █▃▅▂                          
  ▃▁▁▁▁▁▁▄▄▄▃▅▃▃▄▅▄▄▇▃▄▄▄▄▁▄██▆▆▇█████▅▆▅▅▆▃▁▁▅▁▁▄▃▅▄▅▁▄▃▃▃▃▅ ▃
  27.1 ms         Histogram: frequency by time        38.9 ms <

 Memory estimate: 3.81 MiB, allocs estimate: 2.

In [140]:
function abs_dist(y::Array{T}, X::Array{T}, j) where T
    res = zero(eltype(y))
    @inbounds @simd  for k in eachindex(y)
        res += abs(X[k, j] - y[k])
    end
    return res
end

function linear_scann_exact(dist, query, X)

    n_features, n_examples = size(X)
    distances = Array{Float32}(undef, n_examples)
    
    @inbounds for j in 1:n_examples
        distances[j] = dist(query, X, j)    
    end
    return distances
end

@benchmark linear_scann_exact($abs_dist, $query_code, $PQcodes)

BenchmarkTools.Trial: 446 samples with 1 evaluation.
 Range (min … max):   7.927 ms … 100.532 ms  ┊ GC (min … max): 0.00% … 88.26%
 Time  (median):     10.893 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   11.194 ms ±   5.780 ms  ┊ GC (mean ± σ):  3.42% ±  5.95%

                            ▂ ▅▅▃█▄▃██▆▃▁▁                      
  ▃▁▁▁▁▃▁▃▄▃▇▇█▄▆▄▃▃▃▄▃▃▆▅▆▅███████████████▇▅▇▇▅▃▄▄▄▃▃▄▁▃▃▃▁▃▄ ▄
  7.93 ms         Histogram: frequency by time         13.3 ms <

 Memory estimate: 3.81 MiB, allocs estimate: 2.

In [141]:
randint()

LoadError: UndefVarError: randint not defined